<a href="https://colab.research.google.com/github/Brevex/Hotel-Booking-Demand-Data-Analysis/blob/main/Hotel_Booking_Demand_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Analysis period between July/2015 and August/2017**

The data is originally from the article Hotel Booking Demand Datasets, written by Nuno Antonio, Ana Almeida and Luis Nunes for Data in Brief, Volume 22, February 2019. This dataset contains reservation information for a city hotel and a resort hotel and includes information such as when the reservation was made, length of stay, number of adults, children and/or infants, and number of available parking spaces, among other things .


# **REQUIRED LIBRARIES**

In [46]:
import pandas as pd
import numpy as np
import seaborn as sns

In [47]:
# Sort by month instead of alphabetical order

!pip install -q sort-dataframeby-monthorweek
!pip install -q sorted-months-weekdays

In [48]:
from matplotlib import pyplot as plt
%matplotlib inline

import sort_dataframeby_monthorweek as sd

# **IMPORTING FILE**

In [ ]:
!gdown https://raw.githubusercontent.com/Brevex/Hotel-Booking-Demand-Data-Analysis/main/hotel_data.csv

In [ ]:
# Defining initial data frame (without data processing)

df = pd.read_csv('/content/hotel_data.csv')
df

# **DATA ANALYSIS**

In [ ]:
# Database Body

df.shape

In [ ]:
# Listing the Database columns

columnIndex = 1

for i in df.columns:

    print(columnIndex, ":", i)
    columnIndex += 1

In [ ]:
# Analyzing the data type of each column in the Database

df.info()

In [ ]:
# Checking the percentage of missing values

for analyzedValue in df.columns:
    if df[analyzedValue].isnull().any():

      missingValuesPercentual = (df[analyzedValue].isnull().sum() * 100) / df.shape[0]
      print(f"{analyzedValue}: {missingValuesPercentual:.3f}% of values are null.")

# **DATA CLEANING**

In [ ]:
# Remove the [agent, company] Columns

df.drop(["agent", "company"], axis=1, inplace=True)

In [ ]:
# Fields that contain NULL values and quantity

for nullValues in df.columns:
    if df[nullValues].isnull().any():

        print(nullValues + ":", df[nullValues].isnull().sum())

In [74]:
# Removing "null" values

df.dropna(axis =0 , inplace = True)

In [ ]:
# Descriptive statistical summary

df.describe().T

In [ ]:
df.describe(include = "object").T

# **URBAN HOTEL Vs RESORT**

In [ ]:
# Most reserved type

Text = ['Urban Hotel','Resort']

plt.pie(df["hotel"].value_counts(), labels=Text,autopct = "%0.01f%%")
plt.title("Booking Percentage")

plt.show()

In [ ]:
# Comparison of canceled and non-cancelled bookings

## (1) canceled
## (0) non-cancelled

sns.countplot(data = df, x = 'is_canceled', hue = 'hotel')
plt.title("Comparison of canceled and non-cancelled bookings")

In [ ]:
# Annual evolution of bookings

# Only 2016 has all the months of the year for analysis

sns.countplot(data = df, x = 'arrival_date_year', hue = 'hotel')
plt.title("Annual evolution of bookings")

# **CUSTOMERS ORIGIN**

In [ ]:
# List a ranking of the countries that generate the most customers for hotels

costumersCountry = df[df['is_canceled'] == 0]['country'].value_counts().reset_index()

costumersCountry.columns = ['Country','Costumers']

costumersCountry

# **HIGH TIMES OF THE YEAR**

In [86]:
data_resort = df[(df["hotel"] == "Resort Hotel") & (df["is_canceled"] == 0)]
data_city = df[(df["hotel"] == "City Hotel") & (df["is_canceled"] == 0)]

In [ ]:
resort_hotel=data_resort.groupby(['arrival_date_month'])['adr'].mean().reset_index()
resort_hotel

In [ ]:
city_hotel=data_city.groupby(['arrival_date_month'])['adr'].mean().reset_index()
city_hotel

In [ ]:
final=resort_hotel.merge(city_hotel,on = 'arrival_date_month')
final.columns=['month', 'price_for_resort', 'price_for_city_hotel']

final

In [91]:
def sort_month(df, column_name):
    return sd.Sort_Dataframeby_Month(df, column_name)

In [ ]:
final = sort_month(final,'month')
final

In [ ]:
sns.lineplot(x ='month', y= 'price_for_resort', data=final, label = 'Resort Hotel')
sns.lineplot(x = 'month', y = 'price_for_city_hotel',data=final, label= 'City Hotel')

plt.xticks(rotation = 45)
plt.legend(ncol = 1, loc = "best", frameon = False);

In [ ]:
data = sort_month(df, 'arrival_date_month')

sns.lineplot(data = df, x = "arrival_date_month", y = "is_canceled", hue = "hotel")
plt.title("Average cancellation per month")
plt.xticks(rotation = 45)
plt.legend(ncol = 1, loc = "best", frameon = False);

In [ ]:
# Busiest months in hotels

mesesData = df.groupby(['arrival_date_month','hotel']).size()
mesesData = mesesData.unstack()

mesesData.plot(kind = 'line')

In [99]:
# lead_time: Number of days elapsed between the booking entry date and the check-out date

lead_cancel=df.groupby(['lead_time','hotel']).agg( reservations = ('lead_time', 'size'),
                                                     cancelado_mean = ('is_canceled', 'mean')).reset_index()

In [ ]:
plt.title("Average bookings per check-out period")

sns.regplot(data=lead_cancel.loc[lead_cancel.hotel == 'City Hotel'], x = 'lead_time', y = 'cancelado_mean', label = 'City Hotel')
sns.regplot(data=lead_cancel.loc[lead_cancel.hotel == 'Resort Hotel'], x = 'lead_time', y = 'cancelado_mean', label = 'Resort Hotel')

plt.legend(ncol=1, loc="upper right", frameon = False);

# **CAUSES AND RELATIONS**

In [ ]:
# Heatmap correlation:

# - arrival_date_week_number and arrival_date_year have a negative correlation of -0.54.
# - stays_in_weekend_nights and stays_in_week_nights have a positive correlation of 0.49.
# - is_repeated_guest and previous_bookings_not_canceled have a positive correlation of 0.42.

plt.figure(figsize = (16, 8))

rel = df.corr()

sns.heatmap(rel, cmap = "coolwarm", annot = True)

rel